## Código desenvolvido em Java para criação da aplicação que faz a contagem de hashtags somente de uma determina palavra indicada pelo usuário como parâmetro. Também deleta a pasta destino passada como parametro em caso de a mesma existir. Projeto inicial chamado contapalavraparam contendo a estrutura-base das classes.

### 1-Desenvolvimento das 3 classes Mapper, Reducer e Driver para implementação da aplicação

In [ ]:
// Desenvolvimento da classe Driver ContaPalavraParamDriver.java

import java.io.IOException;
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.FileSystem;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
import org.apache.hadoop.mapreduce.Job;

public class ContaPalavraParamDriver {
	public static void main(String[] args) throws IOException, ClassNotFoundException, InterruptedException{
		Configuration conf = new Configuration();
		//recebendo o parametro palavra do usuario
		conf.set("palavra",args[2]);
		
		//verifica se a pasta destino já existe, se sim, a pasta será excluida
		conf.set("fs.defaultFS","hdfs://localhost:8020/");
		FileSystem dfs = FileSystem.get(conf);
		String nomeDir = args[1];
		Path caminhoDir = new Path(dfs.getWorkingDirectory()+"/"+nomeDir);
		
		if(dfs.exists(caminhoDir)){
			System.out.println("Pasta destino já existe e será excluida");
			dfs.delete(caminhoDir);
		}

		Job job = Job.getInstance(conf);
		job.setJarByClass(ContaPalavraParamDriver.class);
		job.setMapperClass(ContaPalavraParamMapper.class);
		job.setReducerClass(ContaPalavraParamReducer.class);
		job.setOutputKeyClass(Text.class);
		job.setOutputValueClass(IntWritable.class);
		job.setNumReduceTasks(1);

		FileInputFormat.addInputPath(job, new Path(args[0]));
		FileOutputFormat.setOutputPath(job, new Path(args[1]));		
		System.exit(job.waitForCompletion(true) ? 0 : 1);
	}
}

In [ ]:
// Desenvolvimento da classe Mapper ContaPalavraParamMapper.java

import java.io.IOException;
import java.util.StringTokenizer;
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;

public class ContaPalavraParamMapper extends Mapper<Object, Text, Text, IntWritable> {

	private final static IntWritable numeroum = new IntWritable(1);

	public void map(Object key, Text value, Context context) throws IOException, InterruptedException {
		//capturando o parametro palavra informado pelo usuario
		Configuration conf = context.getConfiguration();
		String palavra = conf.get("palavra").toString().toLowerCase();
		
		StringTokenizer itr = new StringTokenizer(value.toString().replaceAll("[^a-zA-Z ]", "").toLowerCase());
		while (itr.hasMoreTokens()) {
			String itr1 = itr.nextToken();
			if(itr1.equals(palavra)){
				context.write(new Text(itr1), numeroum);
			}
			
		}
	}
}

In [ ]:
// Desesenvolvimento da classe Reducer ContaPalavraParamReducer.java

import java.io.IOException;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;

public class ContaPalavraParamReducer extends
		Reducer<Text, IntWritable, Text, IntWritable> {

	public void reduce(Text key, Iterable<IntWritable> values, Context context)
			throws IOException, InterruptedException {
		int soma = 0;
		for (IntWritable val : values) {
			soma += val.get();
		}
		context.write(key, new IntWritable(soma));
	}
}

### 2-Geração do arquivo JAR da aplicação e definição do path:

Exemplo: /home/cloudera/Documents/contapalavraparam.jar

### 3-Execução do job JAR no ambiente Hadoop, consumo do arquivo bases/base_tw.txt já armazenado no HDFS e passado a palavra "bigdata" como parametro  para contagem desta palavra no arquivo:

hadoop jar ~/Documents/contapalavraparam.jar ContaPalavraParamDriver bases/base_tw.txt saida/contapalavraparam bigdata